In [1]:
import sys, os, random, time
from copy import deepcopy
sys.path.append('./model')

# from remiplus.input_representation import remi2midi  # PDMX-TODO
from dataloader import RVQTokensDataset, REMIFullSongTransformerDataset
from model.residualVQ import GPT2TokensGenerator, TransformerGenerator, TransformerResidualVQ
from utils import pickle_load, numpy_to_tensor, tensor_to_numpy, pickle_dump

import torch
import yaml
import numpy as np
from scipy.stats import entropy

/home/jingyue/miniconda3/envs/mmp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_path = 'config/generator_PDMX_balanced.yaml'
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)

device = config['training']['device']

In [3]:
dset_tokens = RVQTokensDataset(
    data_dir=config['data']['data_dir'],
    # pieces=pickle_load(config['data']['test_split']),
    pieces=pickle_load(config['data']['train_split']),
    model_max_bars=config['generate']['max_bars'],
    num_tokens=config['data']['num_quantizers'],
    codebook_size=config['data']['codebook_size'],
    shuffle=False,
    first_token_only=config['data']['first_token_only'],
    first_token_first=config['data']['first_token_first']
    )
    

[preparing data] queue generated


In [4]:
mconf = config['model']
generator_path = 'ckpt/gen_12L_8H_512D_128E-16_bars_1280-pdmx-density/params/step_17000-RC_5.714-model.pt'

generator = GPT2TokensGenerator(
    mconf['dec_n_layer'], mconf['dec_n_head'], mconf['dec_d_model'], mconf['dec_d_ff'],
    mconf['d_embed'], dset_tokens.vocab_size, config['data']['max_bars'], use_bar_emb=mconf['use_bar_emb']
).to(device)
generator.eval()
generator.load_state_dict(torch.load(generator_path, map_location='cpu'))
print('[info] load generator')

/tmp/ipykernel_3670210/2469001052.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(generator_path, map_location='cpu'))


[info] load generator


In [ ]:
n_pieces = 1
primer_n_bar = 8
num_tokens = config['generate']['num_quantizers']
max_bars = config['generate']['max_bars']

pieces = random.sample(range(len(dset_tokens)), n_pieces)
for p in range(n_pieces):
    p_data = dset_tokens[pieces[p]]
    p_prompt = p_data['dec_input'][:(1 + primer_n_bar * num_tokens + 2)]
    p_next = p_data['dec_input'][1 + primer_n_bar * num_tokens + 2]
    print(p_next)
    
    bar_pos = np.concatenate(([max_bars], np.repeat(np.arange(max_bars), num_tokens)))
    
    dec_input = numpy_to_tensor([p_prompt], device=device).permute(1, 0).long()
    dec_bar = numpy_to_tensor([bar_pos[:len(p_prompt)]], device=device).permute(1, 0).long()

    # sampling
    with torch.no_grad():
        logits = generator.generate(dec_input, dec_bar)[0]
    prob = torch.softmax(logits, 0)

    predicted_k_indexes = torch.topk(prob,k=20)
    print(torch.sum(predicted_k_indexes[0]))
    
predicted_k_indexes
    

2334
tensor(0.6826, device='cuda:4')


torch.return_types.topk(
values=tensor([0.3848, 0.1472, 0.0407, 0.0154, 0.0141, 0.0131, 0.0076, 0.0071, 0.0061,
        0.0060, 0.0057, 0.0050, 0.0049, 0.0042, 0.0038, 0.0036, 0.0034, 0.0034,
        0.0033, 0.0032], device='cuda:4'),
indices=tensor([2219, 2936, 2471, 2140, 2916, 2454, 2497, 2984, 2181, 2182, 2723, 2201,
        2549, 2494, 2555, 2073, 2827, 2950, 2975, 2049], device='cuda:4'))

: 